# Парсинг данных. Практика 3

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
from time import sleep
from tqdm import tqdm

## Задание

Дана ссылка на первую страницу [каталога книжных новинок](https://www.bgshop.ru/catalog/group?id=444&page=1&sort=1&instock=). Соберите информацию о книгах со всех страниц. 

In [2]:
urls = []

for i in range(1, 3):
    urls.append(f'https://www.bgshop.ru/catalog/group?id=444&page={i}&sort=1&instock=')
urls

['https://www.bgshop.ru/catalog/group?id=444&page=1&sort=1&instock=',
 'https://www.bgshop.ru/catalog/group?id=444&page=2&sort=1&instock=']

In [3]:
page = requests.get(urls[0])
page.encoding='utf-8'

In [4]:
soup = BeautifulSoup(page.text)
soup

<!DOCTYPE html>
<html prefix="og: http://ogp.me/ns#">
<head>
<meta charset="utf-8"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<meta content="fd9a658378927472" name="mailru-verification"/>
<title>Книжные новинки | Библио-Глобус</title>
<meta content="Еженедельное обновление новой литературы от крупных издательств. Будь в курсе самых популярных книжных новинок!" name="description"/>
<meta content="width=device-width, initial-scale=1" name="viewport"/>
<meta content="all,follow" name="robots"/>
<link href="/favicon.png" rel="icon" type="image/png"/>
<link href="/favicon.ico" rel="icon" type="image/x-icon"/>
<link href="/searchprovider.xml" rel="search" title="Biblio-Globus" type="application/opensearchdescription+xml"/>
<link href="https://static1.bgshop.ru" rel="dns-prefetch"/>
<link href="https://static2.bgshop.ru" rel="dns-prefetch"/>
<link href="https://fonts.googleapis.com" rel="dns-prefetch"/>
<link href="https://www.google.com" rel="dns-prefetch"/>
<link href="https:/

In [5]:
product = soup.find_all('div', {'class':'product'})[0]
product

<div class="product">
<a class="img_link" href="/product/10967171"><img alt="" class="img-fluid" src="https://static1.bgshop.ru/imagehandler.ashx?fileName=10967171.jpg&amp;width=200"/></a>
<div class="text">
<div class="author">А.Б. Широкорад</div>
<h3><a href="/product/10967171" id="p_title_10967171">Израиль - Палестина. 75 лет великого противостояния</a></h3>
<div class="prices_item">
<div class="price_item_wrapper"><span class="price_item_title">Цена на сайте:</span> <div class="price_item_block"><s>869 ₽</s> <span class="price_item_new price_item_with_discount">782 ₽</span></div></div>
<div class="price_item_wrapper bt-divider"><span class="price_item_title">Цена в магазине:</span> <div class="price_item_block"><span class="price_item_new">869 ₽</span></div></div>
</div>
<p class="status im_status_title">
<span style="color: #3ba155;"> в наличии</span>
<span class="product-qty">9 шт.</span>
</p>
<div class="buttons row">
<div class="col-9 pl-0">
<span class="btn btn-block btn-prima

In [6]:
'https://www.biblio-globus.ru/' + product.find_all('a')[0].get('href')

'https://www.bgshop.ru/product/10967171'

In [7]:
product.find('div', class_="author").text # можно вместо словоря передать как **kwargs (но т.к. class - зарезервированное слово, то в бьютифулсуп можно прописать как class_

'А.Б. Широкорад'

In [18]:
product.find('div', {'class':'author'}).text # такая запись и через кварги выше это одно и то же

'А.Б. Широкорад'

In [8]:
product.find('h3').text

'Израиль - Палестина. 75 лет великого противостояния'

In [9]:
price = product.find('span', class_="price_item_new price_item_with_discount").text
price

'782\xa0₽'

In [10]:
# сразу научимся чистить цену

price_clean = int(re.findall(r'\d+', price)[0])
price_clean

782

In [12]:
qty = soup.find('span', class_='product-qty').text
qty

'9 шт.'

In [13]:
qty_clean = int(re.findall(r'\d+', qty)[0])
qty_clean

9

Спарсим все данные.

In [14]:
books = []

for u in tqdm(urls):
    page = requests.get(u)
    soup = BeautifulSoup(page.text)
    sleep(1)
    info = soup.find_all('div', {'class':'product'})
    for i in info:
        author = i.find('div', class_="author").text
        title = i.find('h3').text
        try:
            price = i.find('span', class_="price_item_new price_item_with_discount").text            
        except:
            price = i.find('span', class_="price_item_new_lg").text # на случай, если цена без скидки
            
        price_clean = int(re.findall(r'\d+', price)[0])    
        qty = soup.find('span', class_='product-qty').text
        qty_clean = int(re.findall(r'\d+', qty)[0])
        url0 = 'https://www.bgshop.ru' + i.find('a').get('href')

        books.append((author, title, price_clean, qty_clean, url0))

100%|████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:04<00:00,  2.23s/it]


In [19]:
df = pd.DataFrame(books)
df.columns = ['Автор', 'Название', 'Цена', 'Количество', 'Ссылка']

In [16]:
df

,Автор,Название,Цена,Количество,Ссылка
0,А.Б. Широкорад,Израиль - Палестина. 75 лет великого противост...,782,9,https://www.bgshop.ru/product/10967171
1,Михалкова Е.И.,"Нежные листья, ядовитые корни",611,9,https://www.bgshop.ru/product/10967057
2,Фернандес А.,Аргентина. Полная история страны,719,9,https://www.bgshop.ru/product/10967017
3,Т. Хайэм,Мир до нас: Новый взгляд на происхождение чело...,782,9,https://www.bgshop.ru/product/10966967
4,Лю Цысинь,Задача трех тел (Воспоминания о прошлом Земли ...,665,9,https://www.bgshop.ru/product/10966879
5,Лайелл Р.,"Путешествия англичанина по России, Крыму, Кавк...",1554,9,https://www.bgshop.ru/product/10966806
6,А.В Курпатов,Самоучитель 3 в 1. Полная версия,1988,9,https://www.bgshop.ru/product/10966704
7,Чжунхо С.,Корея. Полная история,360,9,https://www.bgshop.ru/product/10966534
8,А. Феверс,Огневица,854,9,https://www.bgshop.ru/product/10966475
9,Дайо А.,Неизвестный Наполеон. Эпопея о величии и трагедии,1197,9,https://www.bgshop.ru/product/10965329


In [55]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24 entries, 0 to 23
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Автор       24 non-null     object
 1   Название    24 non-null     object
 2   Цена        24 non-null     int64 
 3   Количество  24 non-null     int64 
 4   Ссылка      24 non-null     object
dtypes: int64(2), object(3)
memory usage: 1.1+ KB


In [20]:
df = df.sort_values('Цена', ascending = False).reset_index(drop=True)

In [57]:
df

,Автор,Название,Цена,Количество,Ссылка
0,.,Шедевры архитектуры,3111,16,https://www.bgshop.ru/product/10958326
1,О`Брайн П.,Большой исторический атлас мира,2098,16,https://www.bgshop.ru/product/10961445
2,А.В Курпатов,Самоучитель 3 в 1. Полная версия,1988,10,https://www.bgshop.ru/product/10966704
3,,Шедевры архитектуры,1920,16,https://www.bgshop.ru/product/10960747
4,Хабенский К.Ю.,Московский художественный театр. МХТ 125 лет н...,1648,16,https://www.bgshop.ru/product/10955592
5,Джонсон Д.,Дымовое древо,1563,16,https://www.bgshop.ru/product/10961458
6,Лайелл Р.,"Путешествия англичанина по России, Крыму, Кавк...",1554,10,https://www.bgshop.ru/product/10966806
7,Дайо А.,Неизвестный Наполеон. Эпопея о величии и трагедии,1197,10,https://www.bgshop.ru/product/10965329
8,Васильев А.А.,Сокровища кочевника: Париж и далее везде,1197,10,https://www.bgshop.ru/product/10964938
9,Старгардт Н.,Свидетели войны. Жизнь детей при нацистах,1189,16,https://www.bgshop.ru/product/10961713


In [41]:
df.to_excel('bg_books.xlsx', encoded='utf-8', index=False)

TypeError: NDFrame.to_excel() got an unexpected keyword argument 'encoded'

## Задание 2

По аналогии спарсите все книги, в которых есть слово "Python".

In [34]:
# ваш код
p0 = requests.get('https://www.biblio-globus.ru/search?query=Python&page=1')
p0

<Response [200]>

In [39]:
soup = BeautifulSoup(p0.text)
print(soup.prettify())

<!DOCTYPE html>
<html prefix="og: http://ogp.me/ns#">
 <head>
  <meta charset="utf-8"/>
  <meta content="IE=edge" http-equiv="X-UA-Compatible"/>
  <meta content="fd9a658378927472" name="mailru-verification"/>
  <title>
   Результаты поиска: «Python»
  </title>
  <meta content="" name="description"/>
  <meta content="width=device-width, initial-scale=1" name="viewport"/>
  <meta content="all,follow" name="robots"/>
  <link href="/favicon.png" rel="icon" type="image/png"/>
  <link href="/favicon.ico" rel="icon" type="image/x-icon"/>
  <link href="/searchprovider.xml" rel="search" title="Biblio-Globus" type="application/opensearchdescription+xml"/>
  <link href="https://static1.bgshop.ru" rel="dns-prefetch"/>
  <link href="https://static2.bgshop.ru" rel="dns-prefetch"/>
  <link href="https://fonts.googleapis.com" rel="dns-prefetch"/>
  <link href="https://www.google.com" rel="dns-prefetch"/>
  <link href="https://www.googletagmanager.com" rel="dns-prefetch"/>
  <link href="https://www.gst